<a href="https://colab.research.google.com/github/noushadkhan01/web-scraping/blob/master/internshal_internships_web_scrapping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
def clean_text(text):
  import string
  punctuations = string.punctuation.replace('-', '').replace("'", '') + '\n'
  text = [i for i in text if i not in punctuations]
  text = ''.join(text)
  text = text.strip()
  return text

In [0]:
def clean_stipend(text):
  import numpy as np
  text = text.lower().strip()
  text = [i for i in text]
  a = ''
  final_text = ''
  for i in text:
    a += i
    try:
      np.float64(a)
      final_text = a
    except:
      break
  if final_text:
    return np.float64(final_text)
  return np.nan

In [0]:
def clean_apply_by(text):
  import numpy as np
  f = text.find("'")
  text = text[:f] + ' 2020'
  if text == 'Not Provide 2020':
    return np.nan
  return text

In [0]:
def apply_by_to_date(text):
  import numpy as np
  from datetime import datetime
  try:
    a = datetime.strptime(text, '%d %b %Y')
    return a
  except:
    return np.nan

In [0]:
def get_internshala_internships(url, n = 0):
  import numpy as np
  import pandas as pd
  import requests 
  from bs4 import BeautifulSoup
  all_items = []
  no_internship_text = ":( Sorry, We couldn't find internships matching your requirements."
  while True:
    print(f'n is {n}')
    if n:
      url = url + f'/page-{n}'
    try:
      html = requests.get(url)
      soup = BeautifulSoup(html.content, 'lxml')
      all_internship = soup.find_all('div', class_ = 'individual_internship')
      text = all_internship[0].find('h4').text
      if text == no_internship_text:
        print(no_internship_text)
        break
      l = len(all_internship)
      n2 = 0
      for i in all_internship:
        try:
          id = i.get('internshipid')
          location = i.find_all('a', class_ = 'location_link')[0].text
          link = 'https://internshala.com/' + i.find_all('a')[0].get('href')
          td = i.find_all('table')[0].find_all('td')
          start_date = td[0].text
          duration = td[1].text
          stipend = td[2].text
          posted_on = td[3].text
          apply_by = td[4].text
          items = [id, location, start_date, duration, stipend, posted_on, apply_by]
          items = list(map(clean_text, items))
          items.insert(2, link)
          del id, location, link, start_date, duration, stipend, posted_on, apply_by
          all_items.append(items)
          n2 += 1
        except:
          continue
      n += 1
    except Exception as e:
      print(f'{e.__class__.__name__}:--- {e.args[0]}')
      break
  if len(all_items) == 0:
    return no_internship_text
  df = pd.DataFrame(np.array(all_items), columns = ['id', 'location', 'link', 'start_date', 'duration', 'stipend', 'posted_on', 'apply_by'])
  df.drop_duplicates(inplace = True)
  df['stipend'] = df.stipend.apply(clean_stipend)
  df['apply_by'] = df['apply_by'].apply(clean_apply_by)
  df['apply_by'] = df['apply_by'].apply(apply_by_to_date)
  df.dropna(inplace = True)
  return df

In [0]:
url = 'http://internshala.com/internships/data%20science,machine%20learning-internship'

In [142]:
all_internships = get_internshala_internships(url)

n is 0
n is 1
n is 2
n is 3
n is 4
n is 5
n is 6
:( Sorry, We couldn't find internships matching your requirements.


In [143]:
all_internships.head(2)

,id,location,link,start_date,duration,stipend,posted_on,apply_by
0,886862,Mumbai,https://internshala.com//internship/detail/dee...,1 May - 10 Jun'20,6 Months,5000.0,26 Mar'20,2020-04-23
1,886861,Work From Home,https://internshala.com//internship/detail/dee...,Immediately,6 Months,2000.0,26 Mar'20,2020-04-23


In [144]:
all_internships.shape

(156, 8)

In [0]:
all_internships.sort_values(by = 'apply_by', inplace = True)

In [146]:
all_internships

,id,location,link,start_date,duration,stipend,posted_on,apply_by
171,872995,Mumbai,https://internshala.com//internship/detail/dat...,Immediately,2 Months,5000.0,9 Mar'20,2020-03-31
166,873937,Delhi,https://internshala.com//internship/detail/mac...,Immediately,6 Months,25000.0,11 Mar'20,2020-04-09
165,874995,Bangalore,https://internshala.com//internship/detail/mac...,Immediately,3 Months,2000.0,11 Mar'20,2020-04-09
164,875255,Work From Home,https://internshala.com//internship/detail/bus...,Immediately,2 Months,5000.0,11 Mar'20,2020-04-09
163,874764,Delhi,https://internshala.com//internship/detail/bus...,Immediately,2 Months,9000.0,11 Mar'20,2020-04-09
...,...,...,...,...,...,...,...,...
155,876282,Jaipur,https://internshala.com//internship/detail/mar...,Immediately,6 Months,7000.0,12 Mar'20,2020-05-01
148,877519,Kurnool,https://internshala.com//internship/details/da...,Immediately,3 Months,12500.0,15 Mar'20,2020-05-04
149,877484,Kurnool,https://internshala.com//internship/details/ma...,Immediately,3 Months,10000.0,15 Mar'20,2020-05-04
145,878183,Delhi,https://internshala.com//internship/detail/bus...,Immediately,6 Months,3000.0,16 Mar'20,2020-05-05


In [147]:
all_internships.link.values[0]

'https://internshala.com//internship/detail/data-analytics-internship-in-mumbai-at-nymbl-technologies-private-limited1583480135'

In [0]:
useful = all_internships[all_internships.apply_by <= '2020-08-04']
useful.sort_values(by = 'stipend', inplace = True, ascending = False)

In [151]:
useful.head(2)

,id,location,link,start_date,duration,stipend,posted_on,apply_by
189,864018,Hyderabad,https://internshala.com//internship/detail/dat...,Immediately,6 Months,25000.0,27 Feb'20,2020-04-17
119,882223,Work From Home,https://internshala.com//internship/details/ta...,Immediately,6 Months,25000.0,18 Mar'20,2020-04-16


In [155]:
a = useful.apply_by.values[0]
a

numpy.datetime64('2020-04-17T00:00:00.000000000')

In [154]:
for i in useful.index:
  print(useful.location.loc[i], useful.link.loc[i],'\n', useful.stipend.loc[i], useful.apply_by.loc[i])
  print('\n', '***'*30)

Hyderabad https://internshala.com//internship/detail/data-analysis-internship-in-hyderabad-at-corteva-agri-science-the-agriculture-division-of-dowdupont1582737905 
 25000.0 2020-04-17 00:00:00

 ******************************************************************************************
Work From Home https://internshala.com//internship/details/tableau-development-work-from-home-job-internship-at-datumex-infotech-private-limited1584541527 
 25000.0 2020-04-16 00:00:00

 ******************************************************************************************
Bangalore https://internshala.com//internship/detail/data-analytics-internship-in-bangalore-at-cleantech-solar1585306937 
 25000.0 2020-04-25 00:00:00

 ******************************************************************************************
Kelambakkam https://internshala.com//internship/detail/programming-big-data-and-python-internship-in-multiple-locations-at-sterling-holidays1584084598 
 25000.0 2020-04-11 00:00:00

 *********